# Acute Chronic Master

In [ ]:
import psycopg2
import pandas as pd
from sshtunnel import SSHTunnelForwarder

# EC2 instance connection details
ec2_host = '65.1.183.184'
ec2_username = 'ubuntu'
ec2_pem_key_path = "C:/Users/kumar.rohit.KUMARROHIT-LPT/Desktop/emami-prod.pem"

# AWS RDS PostgreSQL database connection details
rds_host = 'emami-fr-prod-db.crknf6guwalh.ap-south-1.rds.amazonaws.com'
rds_port = 5432
rds_database = 'efrprod'
rds_user = 'emamireaduser'
rds_password = 'emamireadaccess'

try:
    # Create an SSH tunnel to the EC2 instance
    with SSHTunnelForwarder(
        (ec2_host, 22),
        ssh_username=ec2_username,
        ssh_pkey=ec2_pem_key_path,
        remote_bind_address=(rds_host, rds_port),
        local_bind_address=('localhost', 22)
    ) as tunnel:
        # Establish a connection to the RDS PostgreSQL database
        connection = psycopg2.connect(
            host='localhost',
            port=tunnel.local_bind_port,
            database=rds_database,
            user=rds_user,
            password=rds_password
        )
        # Chronic Data Query
        chronic_query = '''
        SELECT DISTINCT
            o.user_id,
            ip2.property_value AS actute_chronic
        FROM 
            line_items 
            LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
            LEFT JOIN orders o ON o.id = osl.order_id
            LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
        WHERE 
            line_items.order_id IN ( 
                SELECT order_id 
                FROM order_status_logs AS ols
                LEFT JOIN orders ON orders.id = ols.order_id
                WHERE 
                    state_changed_on >= '2021-12-31 18:30:00.000000' 
                    AND to_state = 3
                    AND city_id = 13
            )
            AND osl.to_state = 3 AND ip2.property_value = 'Chronic'
        '''

        # Acute Data Query
        acute_query = '''
        SELECT DISTINCT
            o.user_id,
            ip2.property_value AS actute_chronic
        FROM 
            line_items 
            LEFT JOIN order_status_logs osl ON osl.order_id = line_items.order_id
            LEFT JOIN orders o ON o.id = osl.order_id
            LEFT JOIN item_properties ip2 ON ip2.item_id = line_items.variant_id AND ip2.property_id IN ('1352')
        WHERE 
            line_items.order_id IN ( 
                SELECT order_id 
                FROM order_status_logs AS ols
                LEFT JOIN orders ON orders.id = ols.order_id
                WHERE 
                    state_changed_on >= '2021-12-31 18:30:00.000000' 
                    AND to_state = 3
                    AND city_id = 13
            )
            AND osl.to_state = 3 AND ip2.property_value = 'Acute'
        '''

        # Fetch DataFrames
        chronic_df = pd.read_sql(chronic_query, connection)
        acute_df = pd.read_sql(acute_query, connection)
        
        # Identify users exclusive to the acute dataset
        acute_only_df = acute_df[~acute_df['user_id'].isin(chronic_df['user_id'])]

        # Combine acute-only data with the full chronic data
        combined_df = pd.concat([acute_only_df, chronic_df], ignore_index=True)

        # Save to Excel with one sheet
        with pd.ExcelWriter('acute_chronic_master.xlsx') as writer:
            combined_df.to_excel(writer, sheet_name='acute_chronic_master', index=False)

        print("Data saved to acute_chronic_master.xlsx")

except (Exception, psycopg2.Error) as error:
    print("Error connecting to PostgreSQL:", error)
finally:
    # Close the database connection
    if connection:
        connection.close()
        print("PostgreSQL connection is closed")

print("Updated at", pd.Timestamp.now())